# Drive time analysis for opioid epidemic

This notebook performs drive time analysis for clinics of opioid epidemic treatment/prevention/both in the county of Oakland, Michigan. 

### 1. Reading in clinics and Oakland county census block group (CBG) boundaries

In [31]:
import arcgis
from datetime import datetime
from arcgis.features import FeatureLayer

In [9]:
# Layer URLS for the Opioid Epidemic Clinics in Oakland County and Census Block Groups of Oakland County
clinic_url = 'https://services8.arcgis.com/TWfU0bgvgUkCgHLa/arcgis/rest/services/Oakland_County_Licensed_Program_Listings/FeatureServer/0/'
oakland_cbg = 'https://services.arcgis.com/bkrWlSKcjUDFDtgw/arcgis/rest/services/Oakland_2010_census_bg/FeatureServer/0/'

# create feature layers
clinic_layer = FeatureLayer(clinic_url)
oakland_layer = FeatureLayer(oakland_cbg)

In [10]:
# Query all clinics and census block groups as FeatureSets
clinic_features = clinic_layer.query()
print('Total number of rows in the clinic dataset: ')
print(len(clinic_features.features))

oakland_features = oakland_layer.query()
print('Total number of rows in the oakland dataset: ')
print(len(oakland_features.features))

Total number of rows in the clinic dataset: 
122
Total number of rows in the oakland dataset: 
934


In [11]:
# Convert to SpatialDataFrames
clinic = clinic_features.sdf
clinic.head()

,Address,City,Contact_Name,FID,Key_,LicenseNumber,Name,Phone_Number,SHAPE,State,Type,Zip
0,"989 University Dr., Ste. 108",Pontiac,"Dr. Carmen Cardona, Program Director",1,Case Management; Integrated Treatment; Outpa...,# SA0631356,SUNRISE TREATMENT CENTER,248-481-2531,"{""x"": -9268869.4566, ""y"": 5259550.238600001, ""...",MI,Treatment,48342
1,"4120 W. Maple Rd., Ste. 201",Bloomfield Hills,"Stacy Gutowsky, Program Director",2,Alcohol Hwy Safety Education; Community Change...,# SA0631330,"EIP, INC. (Education Intervention Programs)",248-693-0336,"{""x"": -9271532.8795, ""y"": 5242995.079599999, ""...",MI,Prevention,48301
2,3283 Bloomfield Park Dr,West Bloomfield,"Karen Fraiberg, Program Director",3,"Case Management; Community Change, Alternative...",# SA0631364,"THE ART OF HOPE, HEALING & HAPPINESS",248-229-4717,"{""x"": -9278211.6833, ""y"": 5246510.740000002, ""...",MI,Both,48323
3,2710 W. 12 Mile Rd.,Berkley,"Kathy Smaller, Program Director",4,Case Management; Early Intervention; Integrate...,# SA0631165,"RECOVERY CONSULTANTS, INC.",248-543-1090,"{""x"": -9260261.7618, ""y"": 5236694.745800003, ""...",MI,Treatment,48072
4,"3139 W. Huron St. Waterford, MI 48328",Waterford,"Kathy Smaller, Program Director",5,Case Management; Early Intervention; Integrate...,# SA0631166,"RECOVERY CONSULTANTS, INC.",248-738-8400,"{""x"": -9278492.0901, ""y"": 5256952.089100003, ""...",MI,Treatment,48328


In [12]:
oakland = oakland_features.sdf
oakland.head()

,ACRES,CNTY_CODE,FID,LABEL,LAYOUT,LINK,NAME,PENINSULA,SHAPE,SQKM,SQMILES,Shape__Area,Shape__Length,TYPE,VER
0,4075.858559,125,1,BG 1,portrait,1251200001,1200001,lower,"{""rings"": [[[-9250818.94177608, 5289778.006450...",16.494592,6.368568,3.073481e+07,22611.540412,BG,14a
1,8797.887541,125,2,BG 1,portrait,1251203001,1203001,lower,"{""rings"": [[[-9264026.53064651, 5290079.339554...",35.604171,13.746784,6.633129e+07,35417.658796,BG,14a
2,6693.987915,125,3,BG 1,landscape,1251210001,1210001,lower,"{""rings"": [[[-9270503.93750677, 5288614.193867...",27.089900,10.459421,5.047787e+07,29332.713639,BG,14a
3,5399.750207,125,4,BG 1,portrait,1251214001,1214001,lower,"{""rings"": [[[-9269511.1796278, 5286236.4642479...",21.852249,8.437162,4.070574e+07,29772.396132,BG,14a
4,3719.286317,125,5,BG 2,portrait,1251222002,1222002,lower,"{""rings"": [[[-9281344.03654134, 5290365.890503...",15.051580,5.811421,2.804448e+07,23245.466116,BG,14a


### 2. Drive time analysis

In [19]:
# Connect to GIS
from arcgis.gis import GIS
gis = GIS(profile="your_online_profile") 

Service areas around the clinics need to be generated twice for two different sets of drive time inputs:

> 1. __`[5, 10, 15, 20, 25]`__ that is needed to get the drive time required from each CBG to the closest clinic
> 2. __`[5]`__ that is needed purely for visualization purposes

Service area generation for the [5, 10, 15, 20, 25] minutes layer

In [21]:
# Generate service areas
result = arcgis.features.use_proximity.create_drive_time_areas(clinic_url, [5, 10, 15, 20, 25], output_name="DriveTimeToClinics_All_2"+str(datetime.now().microsecond), overlap_policy='Dissolve')

In [22]:
# Check type of result
type(result)

arcgis.gis.Item

In [23]:
# Share results for public consumption
result.share(everyone=True)

{'results': [{'itemId': '40bc31789f764184947dbb1da7675889',
   'success': True,
   'notSharedWith': []}]}

In [24]:
# Convert it to a FeatureSet
result_url = str(result.url)+'/0/'
drive_times_all_layer = FeatureLayer(result_url)
driveAll_features = drive_times_all_layer.query()
print('Total number of rows in the service area (upto 25 mins) dataset: ')
print(len(driveAll_features.features))

Total number of rows in the service area (upto 25 mins) dataset: 
5


In [25]:
# Get results as a SpatialDataFrame
driveAll = driveAll_features.sdf
driveAll.head()

,AnalysisArea,FacilityOID,FromBreak,Name,OBJECTID,SHAPE,ToBreak
0,508.239318,None,20,20 - 25,1,"{""rings"": [[[-9332991.0914, 5246110.1884], [-9...",25
1,452.186719,None,15,15 - 20,2,"{""rings"": [[[-9320991.3273, 5233355.6918], [-9...",20
2,426.955043,None,10,10 - 15,3,"{""rings"": [[[-9317466.3064, 5285772.2205], [-9...",15
3,392.647341,None,5,5 - 10,4,"{""rings"": [[[-9293641.3093, 5272963.9065], [-9...",10
4,345.389051,None,0,0 - 5,5,"{""rings"": [[[-9249366.647, 5243768.4208], [-92...",5


#### Service area generation for the visible [5] minutes layer

In [27]:
# Generate service areas
visible_result = arcgis.features.use_proximity.create_drive_time_areas(clinic_url, [5], output_name="DriveTimeToClinics_5mins"+str(datetime.now().microsecond), overlap_policy='Dissolve')

In [28]:
# Grant 'visible_result' public access
visible_result.share(everyone=True)

{'results': [{'itemId': '89166eeecc79414391f2e25078ffa835',
   'success': True,
   'notSharedWith': []}]}

In [29]:
# Generate the URL for this layer (later used for visualization)
visible_result_url = str(visible_result.url)+'/0'
visible_result_url

'https://services7.arcgis.com/JEwYeAy2cc8qOe3o/arcgis/rest/services/DriveTimeToClinics_5mins_2/FeatureServer/0'

### 3. Spatially join the `result_url` layer with `oakland_cbg` layer to obtain drive times for each census block group

In [32]:
# Perform spatial join between CBG layer and the service areas created for all time durations
cbg_with_driveTime = arcgis.features.analysis.join_features(oakland_cbg, result_url, spatial_relationship='Intersects', output_name='Oakland_CBG_allDriveTimes'+str(datetime.now().microsecond))

In [33]:
# Grant 'cbg_with_driveTime' public access
cbg_with_driveTime.share(everyone=True)

{'results': [{'itemId': '4a9bfa6798584c968c29f01a02f14db4',
   'success': True,
   'notSharedWith': []}]}

In [34]:
# Converting it to a FeatureSet
cbg_with_driveTime_url = str(cbg_with_driveTime.url)+'/0/'
drive_times_cbg = FeatureLayer(cbg_with_driveTime_url)
drive_times_features = drive_times_cbg.query()
print('Total number of rows in the drive time (upto 25 mins) dataset: ')
print(len(drive_times_features.features))

Total number of rows in the drive time (upto 25 mins) dataset: 
934


In [35]:
# Convert it to a SpatialDataFrame
drive_time_cbg_df = drive_times_features.sdf
drive_time_cbg_df.head()

,ACRES,AnalysisArea,CNTY_CODE,FacilityOID,FromBreak,Join_Count,LABEL,LAYOUT,LINK,NAME,...,OBJECTID,PENINSULA,SHAPE,SQKM,SQMILES,Shape__Area,Shape__Length,TYPE,ToBreak,VER
0,4075.858559,508.239318,125,None,20,2,BG 1,portrait,1251200001,1200001,...,1,lower,"{""rings"": [[[-9250818.9418, 5289778.0065], [-9...",16.494592,6.368568,3.073481e+07,22611.540412,BG,25,14a
1,8797.887541,508.239318,125,None,20,4,BG 1,portrait,1251203001,1203001,...,2,lower,"{""rings"": [[[-9264026.5306, 5290079.3396], [-9...",35.604171,13.746784,6.633129e+07,35417.658796,BG,25,14a
2,6693.987915,452.186719,125,None,15,3,BG 1,landscape,1251210001,1210001,...,3,lower,"{""rings"": [[[-9270503.9375, 5288614.1939], [-9...",27.089900,10.459421,5.047787e+07,29332.713639,BG,20,14a
3,5399.750207,426.955043,125,None,10,3,BG 1,portrait,1251214001,1214001,...,4,lower,"{""rings"": [[[-9269511.1796, 5286236.4643], [-9...",21.852249,8.437162,4.070574e+07,29772.396132,BG,15,14a
4,3719.286317,452.186719,125,None,15,3,BG 2,portrait,1251222002,1222002,...,5,lower,"{""rings"": [[[-9281344.0365, 5290365.8905], [-9...",15.051580,5.811421,2.804448e+07,23245.466116,BG,20,14a


### 4. Enrich census block groups with suitable data

The 3 geo-enrichment factors that we will be including in this study are:

> 1. 2017 Total Population (Analysis Variable - TOTPOP_CY)
> 2. 2017 Health Insurance (Analysis Variable - X8002_X)
> 3. 2017 Education : Bachelor's Degree (Analysis Variable - BACHDEG_CY)

In [37]:
# Perform data enrichment
enriched_drive_time_cbg = arcgis.features.enrich_data.enrich_layer(drive_times_cbg, analysis_variables=['TOTPOP_CY', 'X8002_X', 'BACHDEG_CY'], output_name="Enriched_DriveTime_CBG"+str(datetime.now().microsecond))

In [38]:
# Check type of 'enriched_drive_time_cbg'
type(enriched_drive_time_cbg)

arcgis.gis.Item

In [39]:
# Grant 'enriched_drive_time_cbg' public access
enriched_drive_time_cbg.share(everyone=True)

{'results': [{'itemId': 'e1904c9cc21f4e1bb3d5b671ea99e9e1',
   'success': True,
   'notSharedWith': []}]}

In [40]:
# Convert it to a Feature Set
enriched_url = str(enriched_drive_time_cbg.url)+'/0/'
enriched_driveTimes_layer = FeatureLayer(enriched_url)
enriched_driveTimes_features = enriched_driveTimes_layer.query()
print('Total number of rows in the drive time (upto 25 mins) dataset: ')
print(len(enriched_driveTimes_features.features))

Total number of rows in the drive time (upto 25 mins) dataset: 
934


In [41]:
# Convert it to a SpatialDataFrame
enriched_df = enriched_driveTimes_features.sdf
enriched_df.head()

,ACRES,AnalysisArea,BACHDEG_CY,CNTY_CODE,ENRICH_FID,FacilityOID,FromBreak,HasData,ID,Join_Count,...,Shape__Length,TOTPOP_CY,TYPE,ToBreak,VER,X8002_X,aggregationMethod,apportionmentConfidence,populationToPolygonSizeRating,sourceCountry
0,4075.858559,508.239318,98,125,1,None,20,1,0,2,...,22611.540412,862,BG,25,14a,1535591,BlockApportionment:US.BlockGroups,2.576,2.191,US
1,8797.887541,508.239318,258,125,2,None,20,1,1,4,...,35417.658796,1617,BG,25,14a,3183423,BlockApportionment:US.BlockGroups,2.576,2.191,US
2,6693.987915,452.186719,74,125,3,None,15,1,2,3,...,29332.713639,653,BG,20,14a,1323720,BlockApportionment:US.BlockGroups,2.576,2.191,US
3,5399.750207,426.955043,909,125,4,None,10,1,3,3,...,29772.396132,4224,BG,15,14a,8447741,BlockApportionment:US.BlockGroups,2.576,2.191,US
4,3719.286317,452.186719,124,125,5,None,15,1,4,3,...,23245.466116,1691,BG,20,14a,2553739,BlockApportionment:US.BlockGroups,2.576,2.191,US


In [42]:
# Print column names for ease of use during visualization
enriched_df.columns

Index(['ACRES', 'AnalysisArea', 'BACHDEG_CY', 'CNTY_CODE', 'ENRICH_FID',
       'FacilityOID', 'FromBreak', 'HasData', 'ID', 'Join_Count', 'LABEL',
       'LAYOUT', 'LINK', 'NAME', 'Name_1', 'OBJECTID', 'PENINSULA', 'SHAPE',
       'SQKM', 'SQMILES', 'Shape__Area', 'Shape__Length', 'TOTPOP_CY', 'TYPE',
       'ToBreak', 'VER', 'X8002_X', 'aggregationMethod',
       'apportionmentConfidence', 'populationToPolygonSizeRating',
       'sourceCountry'],
      dtype='object')

In [43]:
# Format URL for visualization
enriched_lyr_url = enriched_url[:-1]
enriched_lyr_url

'https://services7.arcgis.com/JEwYeAy2cc8qOe3o/arcgis/rest/services/Enriched_DriveTime_CBG316797/FeatureServer/0'

### 5. Visualize results

In [ ]:
# Create map of Oakland county
map1 = gis.map('Oakland, Michigan', zoomlevel=10)
map1

![pythonresultmap](https://user-images.githubusercontent.com/13968196/36760882-94954672-1bea-11e8-94fd-6f68aaee313a.PNG)

In [45]:
# Add next layer for each CBG viualized based on its health insurance spending (X8002_2)
map1.add_layer({"type":"FeatureLayer",
                "url": enriched_lyr_url,
                "renderer":"ClassedSizeRenderer",
                "field_name":"X8002_X",
                "opacity":0.6
               })

In [46]:
# Add layer for service areas reachable within a 5 min drive time of the clinics
map1.add_layer({"type":"FeatureLayer",
                "url": visible_result_url,
                "opacity":0.75
               })

In [47]:
# Add layer of clinics
map1.add_layer({"type":"FeatureLayer",
                "url": clinic_url,
                "opacity":0.75
               })

Similar maps can be generated for the following variables too
> 1. Travel Time (`ToBreak`)
> 2. Population (`TOTPOP_CY`)
> 3. Citizens with Bachelor's Degree (`BACHDEG_CY`)
